In [1]:
from bs4 import BeautifulSoup as bs
import requests

In [2]:
link = 'https://www.amazon.in/Sony-PS5-PlayStation-Console/dp/B0BRCP72X8/ref=zg-bs_videogames_sccl_2/261-1038517-4763225?pd_rd_w=6drTE&content-id=amzn1.sym.7dd29d48-66c1-486c-967d-2ed40101f2ea&pf_rd_p=7dd29d48-66c1-486c-967d-2ed40101f2ea&pf_rd_r=XYSX6EKQ5FCNQP8MAS02&pd_rd_wg=P6CXw&pd_rd_r=8866e043-8660-4c3a-9a42-f03eac9eb197&pd_rd_i=B0BRCP72X8&psc=1'

In [3]:
page = requests.get(link)

In [4]:
page

<Response [503]>

In [6]:
page.content

b'<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->\n<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->\n<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="a-no-js" lang="en-us"><!--<![endif]--><head>\n<meta http-equiv="content-type" content="text/html; charset=UTF-8">\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title dir="ltr">Amazon.in</title>\n<meta name="viewport" content="width=device-width">\n<link rel="stylesheet" href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css">\n<script>\n\nif (true === true) {\n    var ue_t0 = (+ new Date()),\n        ue_csm = window,\n        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },\n        ue_furl = "fls-eu.amazon.in",\n        ue_mid = "A